In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr
import matplotlib.pyplot as plt

In [ ]:
folder=r'D:\Jupyter Notebook\Driver Drowsiness Dataset (DDD)' 

In [3]:
img_class=os.listdir(folder)

In [4]:
ext=['jpeg','jpg','bmp','png'] 

In [ ]:
for i_c in os.listdir(folder):
    class_path = os.path.join(folder, i_c)
    if os.path.isdir(class_path):
        for img in os.listdir(class_path):
            img_path = os.path.join(class_path, img)
            if imghdr.what(img_path) not in ext:
                os.remove(img_path)
                print(f'Deleted {img_path}')


In [ ]:
data=tf.keras.utils.image_dataset_from_directory(r'D:\Jupyter Notebook\Driver Drowsiness Dataset (DDD)') #

In [ ]:
data_iterator=data.as_numpy_iterator()

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [ ]:
batch=data_iterator.next()

In [ ]:
plt.imshow(batch[0][31].astype(int))

In [ ]:
for i in range(0,32):
    plt.imshow(batch[0][i].astype(int))
    plt.show()

In [12]:
data = data.map(lambda x, y: (x/255, y))

In [13]:
scaled_iterator=data.as_numpy_iterator()

In [14]:
batch=scaled_iterator.next()

In [ ]:
for i in range(0,32):
    plt.imshow(batch[0][i])
    plt.show()

In [ ]:
len(data)

In [17]:
train_size = int(0.7 * len(data))
valid_size = int(0.2 * len(data))

train = data.take(train_size)
valid = data.skip(train_size).take(valid_size)
test = data.skip(train_size + valid_size)


In [18]:
from tensorflow import keras
from tensorflow.keras import Sequential

In [19]:
model=Sequential()

In [ ]:
model.add(keras.layers.Conv2D(16,(3,3),1,input_shape=(256,256,3),activation='relu'))
model.add(keras.layers.MaxPooling2D())

model.add(keras.layers.Conv2D(32,(3,3),1,activation='relu'))
model.add(keras.layers.MaxPooling2D())

model.add(keras.layers.Conv2D(16,(3,3),1,activation='relu'))
model.add(keras.layers.MaxPooling2D())

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 16)          │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 14400)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14400)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       3,686,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,696,625 (14.10 MB)

 Trainable params: 3,696,625 (14.10 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss='binary_crossentropy'
)

In [23]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # this function is used to validate and check accuracy simultaneously

In [ ]:
hist = model.fit(train, epochs=5, validation_data=valid, callbacks=[tensorboard_callback]) 

In [25]:
import matplotlib.pyplot as plt

In [ ]:
epochs=range(0,len(hist.history['accuracy']))
plt.plot(epochs,hist.history['val_accuracy'])
plt.plot(epochs,hist.history['accuracy'])
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()
plt.plot(epochs,hist.history['val_loss'])
plt.plot(epochs,hist.history['loss'])
plt.xlabel("epochs")
plt.ylabel("loss")


In [28]:
from tensorflow.keras.models import load_model

In [ ]:
model_dir = os.path.join('models', 'drowsy_driver.h5')
os.makedirs(os.path.dirname(model_dir), exist_ok=True)
model.save(model_dir)

In [ ]:
import numpy as np

test_data_with_labels = test.map(lambda x, y: (x, y))  
y_true = []
y_pred = []

for images, labels in test_data_with_labels:
    preds = model.predict(images)  
    preds = (preds > 0.5).astype(np.int32)
    y_pred.extend(preds.flatten())  
    y_true.extend(labels.numpy())  


In [ ]:
pip show sklearn

In [38]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print("Confusion matrix")
print(cm)


Confusion matrix
[[2259    2]
 [   3 1929]]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

labels = ['Negative', 'Positive']

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)

plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')


plt.show()


In [ ]:
img = cv2.imread('A1.png')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [65]:

import numpy as np

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat<0.5:
    print(" Drowsy Driver")
else:
    print("Non Drowsy Driver")

In [ ]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model(r'D:\Jupyter Notebook\models\drowsy_driver.h5')
model.summary()

In [ ]:
import cv2
import matplotlib.pyplot as plt
img=cv2.imread('s1.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if yhat<0.5:
    print(" Drowsy Driver")
else:
    print("Non Drowsy Driver")